In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install lion-pytorch

In [4]:
%pip install torchmetrics

In [5]:
import torch
from torch import optim
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from lion_pytorch import Lion
from utility.step_lr import StepLR


In [6]:
def train(swa_start, model, swa_model, epoch, device, train_loader, optimizer, log, scheduler, swa_scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.get_last_lr()[0])

In [7]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [8]:
if __name__ == "__main__":

    num_epochs = 50
    # model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'widerestnet-swa'
    # model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.09)

    swa_model = torch.optim.swa_utils.AveragedModel(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80)
    swa_start = 25
    swa_scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.1)

    for epoch in range(num_epochs):
        train(swa_start, model, swa_model, epoch, device, dataset.train, optimizer, log, scheduler, swa_scheduler)
        test(model, device, dataset.valid, log)
        if epoch > swa_start:
          swa_model.update_parameters(model)
          swa_scheduler.step()
        else:
          scheduler.step()

    log.flush()
    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')

100
300
880
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3565  │      0.56 %  ┃   1.000e-02  │       00:04  ┃      0.3674  │     32.00 %  ┃
┃           1  ┃      0.3348  │      0.70 %  ┃   9.996e-03  │       00:00  ┃      0.3623  │     58.33 %  ┃
┃           2  ┃      0.3127  │      0.79 %  ┃   9.985e-03  │       00:00  ┃      0.3573  │     63.67 %  ┃
┃           3  ┃      0.2949  │      0.78 %  ┃   9.965e-03  │       00:00  ┃      0.3504  │     67.00 %  ┃
┃           4  ┃      0.2767  │      0.83 %  ┃   9.938e-03  │       00:00  ┃      0.3437  │     69.33 %  ┃
┃           5  ┃      0.2549  │      0.84 %  ┃   9.904e-03  │       00:00  ┃      0.3380  │     67.67 %  ┃
┃           6  ┃      0.2357  │      0.84 %  ┃   9.862e-03  │       00:00  ┃      0.3277  │     70.67 %  ┃
┃           7  ┃      0.2202  │      0.84 %  ┃   9.812e-03  │       00:00  ┃      0.3212  │     70.33 %  ┃
┃           8  ┃      0.2013  │      0.86 %  ┃   9.755e-03  │       00:00  ┃      0.3108  │     73.67 %  ┃
┃           9  ┃      0.1897  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
print(len(inputs))
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

# print(f1)
# print(acc)
# print(precision)
# print(specificity)
# print(recall)


100


In [11]:
len(dataset.train)

9

In [12]:
len(dataset.test_x)

100